In [1]:
#PART1:getting results sequences and forming memory mapped numpy files

In [3]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [4]:
print('Libraries loaded successfully')

Libraries loaded successfully


In [5]:
def preprocessing(fasta_file, temp_file):
    l = 1000
    fp = open(temp_file, 'w')
    
    with open (fasta_file, 'r') as file:
        seq=[]
        seq_id=[]
        for line in file:
            if line.startswith('>'):
                seq_id.append(line.strip('>').strip('\n'))
            else:
                seq.append(line.strip('\n'))
            
    for j, sq in enumerate(seq):   
        if len(sq) <= 1000:
            fp.write('>%s\n'%(seq_id[j]))
            fp.write('%s\n'%(seq[j]))
        else:
            for i in range(0, len(seq)-l+1, 100):
                new_seq_id = '%s_SEPARATED_SEQUENCE_(%s_%s)' % (seq_id[j], i+1, i+l+1)
                new_seq = sq[i:i+l]
                fp.write('>%s\n'%(new_seq_id))
                fp.write('%s\n'%(new_seq))
    
    #input_handle.close()
    fp.close()
    return 

In [6]:
def fill_aa(seq):
    fill_aa_cnt = 1000 - len(seq)
    add_aa_seq = '_' * fill_aa_cnt
    new_seq = seq + add_aa_seq
    return new_seq

In [7]:
def score_info():
    aa_list = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'X', '_']
    aa_score_info = {}
    for aa in aa_list:
        for aa2 in aa_list:
            if aa == aa2:
                aa_score_info[(aa, aa2)] = 1.0
                aa_score_info[(aa2, aa)] = 1.0
            else:
                aa_score_info[(aa, aa2)] = 0.0
                aa_score_info[(aa2, aa)] = 0.0
    return aa_score_info

In [8]:
def one_hot_encoding(seq, aa_score_info):
    data = np.zeros((1000, 21), dtype=np.float32)
    aa_list = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'X']
    for i, aa in enumerate(seq):
        for j, aa2 in enumerate(aa_list):
            data[i, j] = aa_score_info[aa, aa2]
    return data

In [9]:
counter=0
def create_new_file(file):
    global counter
    counter += 1
    filename = f"{file}/{file}_{counter}.txt"
    return open(filename, 'w')


In [10]:
def run_one_hot_encoding(fasta_file2, temp_file):
    aa_score_info = score_info()
    
    os.makedirs(temp_file)
    
    fp = create_new_file(temp_file)
    
    feature_names = ['Feature%s'%(i) for i in range(1, 21001)] 
    
    fp.write('%s\n'%(','.join(['ID']+feature_names)))
    
    with open (fasta_file2, 'r') as file:
        seq=[]
        seq_id=[]
        for line in file:
            if line.startswith('>'):
                seq_id.append(line.strip('>').strip('\n'))
            else:
                seq.append(line.strip('\n'))
                
                
    for j, sq in enumerate(seq):
        try:
            if len(sq) >= 10 and len(sq) <= 1000:
                if len(sq) < 1000:
                    sq = fill_aa(sq)
                encoded_vector = one_hot_encoding(sq, aa_score_info)
                flatten_encoded_vector = encoded_vector.flatten()
                flatten_encoded_vector_str = [str(each_val) for each_val in flatten_encoded_vector]
                fp.write('%s\n'%(','.join([seq_id[j]]+flatten_encoded_vector_str)))
                
                # Check if the current iteration is a multiple of 1000
                if (j + 1) % 1000 == 0:
                    # If it is, close the current file and create a new one
                    fp.close()
                    fp = create_new_file(temp_file)
                    fp.write('%s\n'%(','.join(['ID']+feature_names)))
            else:
                pass
        except:
            pass
    fp.close()

In [74]:
def create_df(df):
    seq_ids = list(df.index)
    X_temp = df.values
    new_X = []
    for i in range(len(X_temp)):
        temp = np.reshape(X_temp[i], (1000, 21))
        new_X.append(temp)
    
    X = np.asarray(new_X)
    X = X.reshape(X.shape[0], 1000, 21, 1)
    #print('numpy array created')
    return X, seq_ids

In [12]:
#creating dataset
def create_dataset(fasta, tmp_file):
    preprocessing(fasta, 'tmp.txt')
    print('preprocessing done')
    run_one_hot_encoding('tmp.txt', tmp_file)
    print('one hot encoding done')
    return

In [73]:
def create_array(temp_file1, val1):
    
    file_list = os.listdir(temp_file1)
    file_list.sort()
    file_list
    seque_id=[]
    X=np.empty((0, 1000, 21, 1))
    y=np.empty((0,2))
    
    for file in file_list:
        file_path=temp_file1+'/'+file
        temp_df = pd.read_csv(file_path, index_col=0)
        #print('temp df loaded')
        X_new, ids=create_df(temp_df)
        seque_id=seque_id+ids
        X=np.concatenate((X, X_new), axis=0)
        #print('X created')
        y_new=np.full(len(temp_df.index), val1)
        y_new= to_categorical(y_new, 2)
        y=np.concatenate((y, y_new), axis=0)
    
    return X, y, seque_id
    print('done')

In [14]:
print('functions defined')

functions defined


In [92]:
create_dataset('non_GH_of_interest.fasta', 'tmp_non_GH_of_interest')
print('Enzyme dataset loaded')

preprocessing done
one hot encoding done
Enzyme dataset loaded


In [56]:
with open ('tmp.txt', 'r') as file:
    seq=[]
    seq_id=[]
    for line in file:
        if line.startswith('>'):
            seq_id.append(line.strip('>').strip('\n'))

3697

In [93]:
X, y, seq_id=create_array('tmp_non_GH_of_interest',1)
print('enzyme dataset loaded')

enzyme dataset loaded


In [94]:
len(seq_id)

5892

In [95]:
print('saving numpy files')
np.save('X_all_EC',X)
np.save('y_all_EC',y)
print('numpy files saved')

saving numpy files
numpy files saved


In [96]:
X,y=[],[]

In [97]:
#PART-II: testing of the sequences are enzymes or not
from tensorflow.keras.models import load_model
from tensorflow.keras.models import model_from_json

In [98]:
X_test=np.load('X_all_EC.npy')
y_test=np.load('y_all_EC.npy')

In [99]:
yy_df=pd.DataFrame(y_test)
yy=yy_df[1].to_list()
yy.count(1)

5892

In [100]:
model99=load_model('model_09.h5')

In [101]:
y_predicted = model99.predict(X_test)

185/185 [==============================] - 3s 16ms/step


In [102]:
y_predicted_df=pd.DataFrame(y_predicted)
ff=y_predicted_df[1].to_list()
ff

[0.14326594769954681,
 0.14326594769954681,
 0.9783393144607544,
 0.9783393144607544,
 0.9337968230247498,
 0.01943054236471653,
 0.7810900807380676,
 0.8594159483909607,
 0.11847739666700363,
 0.40953555703163147,
 0.4726213216781616,
 0.5180140137672424,
 0.5180140137672424,
 0.996679425239563,
 0.996679425239563,
 0.9649410843849182,
 0.8711735606193542,
 0.6676963567733765,
 0.6268787384033203,
 0.9969075322151184,
 0.9969075322151184,
 0.9969075322151184,
 0.9969075322151184,
 0.9969075322151184,
 0.9969075322151184,
 0.998154878616333,
 0.998154878616333,
 0.998154878616333,
 0.998154878616333,
 0.998154878616333,
 0.998154878616333,
 0.9908440113067627,
 0.9892454743385315,
 0.9908440113067627,
 0.9908440113067627,
 0.9908440113067627,
 0.9908440113067627,
 0.9895245432853699,
 0.991527795791626,
 0.991527795791626,
 0.9994423985481262,
 0.9935817122459412,
 0.9995162487030029,
 0.9995162487030029,
 0.99971604347229,
 0.99971604347229,
 0.9991329312324524,
 0.9991329312324524,
 

In [103]:
len(y_test)

5892

In [104]:
pred_list=[]
for i in ff:
    if i >0.5:
        pred_list.append(1)
    else:
        pred_list.append(0)

In [105]:
pred_list.count(1)

5109

In [106]:
prediction=pd.DataFrame(pred_list)
prediction['ID']=seq_id
prediction

,0,ID
0,0,CUX77898.1&3.2.1.-&GH0
1,0,BAU88504.1&3.2.1.-&GH0
2,1,AAG06182.1&3.2.1.-&GH0
3,1,AAF60322.2&3.2.1.-&GH0
4,1,AAG06872.1&3.2.1.-&GH0
...,...,...
5887,1,AQR80106.1&3.2.1.92&GH171
5888,1,AQR84319.1&3.2.1.92&GH171
5889,1,QJF42043.1&3.2.1.92&GH171
5890,0,BAQ28034.1&4.2.1.179 3.2.1.- 3.2.1.-&GH172


In [107]:
pred=prediction[0].to_list()
pred.count(1)

5109

In [108]:
#saving protein ids that are predicted to be enzyme
enzyme_df=prediction[prediction[0]==1]
enzyme_id=enzyme_df.ID.to_list()
with open('all_non_GH_of_interest_1st_pass.txt','w') as f:
    for id in enzyme_id:
        f.write(id+'\n')